<div style="direction:rtl;text-align:center"><img src="https://dl.mohammadkh.ir/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/></div>
<h1><div style="direction:rtl;text-align:center">Neural Network</div></h1>

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io

### dl & clean & preprocess data

##### dl

In [5]:
path_to_zip = keras.utils.get_file('spa-eng.zip',
                                   origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
                                   extract=True)
path_to_file = os.path.join(os.path.dirname(path_to_zip), 'spa-eng', 'spa.txt')

##### clean

In [16]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_senetence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ", w)       # expand spase ali. --> ali .
    w = re.sub(r'[" "]+', " ", w)             # remove extra space
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)     # revove all - [^a-zA-Z?.~,]
    w = w.rstrip().strip()
    w = '<strat> ' + w + ' <end>'
    return w
print(unicode_to_ascii("Aún no he dicho nada. !&^%"))
print(preprocess_senetence("Aún no he dicho nada. !&^%"))

Aun no he dicho nada. !&^%
<strat> aun no he dicho nada . <end>


In [27]:
def create_dataset(path, num_examples):
    
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    
    word_pairs = [[preprocess_senetence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    
    return zip(*word_pairs)

print(list(create_dataset(path_to_file, 2)))

[('<strat> go . <end>', '<strat> go . <end>'), ('<strat> ve . <end>', '<strat> vete . <end>')]


##### preprocess

In [32]:
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
    # tokenize
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    
    # transfirm input
    tensor = lang_tokenizer.texts_to_sequences(lang)
    
    # pading --> max len
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

# call all fun
def load_dataset(path, num_examples=None):
    
    targ_lang, inp_lang = create_dataset(path, num_examples)
    
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

#decoder
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000)
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
# print(target_lang_tokenizer.word_index)
print(input_tensor[0])
print(convert(input_lang_tokenizer, input_tensor[0]))

[ 1 97  3  2  0  0  0  0  0  0  0  0  0  0  0  0]
1  ....  <strat>
97  ....  ve
3  ....  .
2  ....  <end>
None


#### create dataset

In [37]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

### create and fit model
##### create

In [38]:
example_input_batch, example_target_batch = next(iter(dataset))

In [39]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

simple_hidden = encoder.initilize_hidden_state()
print(simple_hidden[:2])

simple_output, simple_states = encoder(example_input_batch, simple_hidden)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(2, 1024), dtype=float32)


In [40]:
class Attention(keras.layers.Layer):
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
    def call (self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        atteion_weights = tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights
    
attention_layer = Attention(10)

attention_result, attention_weights = attention_layer(simple_hidden, simple_output)

In [41]:
class Decoder(keras.Model):
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights
    
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

x, state, attention_weights = decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

##### fit

In [42]:
optimizer = keras.optimizers.Adam()

loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [43]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [44]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [45]:
EPOCH = 10
for epoch in range(EPOCH):
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        print('Epoch: ', epoch)
        print('Loss: ', batch_loss.numpy())
    checkpoint.save(file_prefix='test1')

Epoch:  0
Loss:  4.7412753
Epoch:  0
Loss:  4.61828
Epoch:  0
Loss:  4.592505
Epoch:  0
Loss:  4.262019
Epoch:  0
Loss:  3.3849387
Epoch:  0
Loss:  4.351911
Epoch:  0
Loss:  3.9540195
Epoch:  0
Loss:  3.3931537
Epoch:  0
Loss:  2.813931
Epoch:  0
Loss:  2.7030475
Epoch:  0
Loss:  2.9139237
Epoch:  0
Loss:  2.9846709
Epoch:  0
Loss:  3.1341822
Epoch:  0
Loss:  3.0347211
Epoch:  0
Loss:  2.9293542
Epoch:  0
Loss:  3.0448067
Epoch:  0
Loss:  3.0080352
Epoch:  0
Loss:  2.8294044
Epoch:  0
Loss:  2.7109988
Epoch:  0
Loss:  2.7461622
Epoch:  0
Loss:  2.7854125
Epoch:  0
Loss:  2.7990637
Epoch:  0
Loss:  2.6540084
Epoch:  0
Loss:  2.6203141
Epoch:  0
Loss:  2.5821288
Epoch:  0
Loss:  2.6138053
Epoch:  0
Loss:  2.7067676
Epoch:  0
Loss:  2.6320775
Epoch:  0
Loss:  2.7107334
Epoch:  0
Loss:  2.5894103
Epoch:  0
Loss:  2.572207
Epoch:  0
Loss:  2.6068554
Epoch:  0
Loss:  2.4366376
Epoch:  0
Loss:  2.4450412
Epoch:  0
Loss:  2.4538991
Epoch:  0
Loss:  2.5021594
Epoch:  0
Loss:  2.590179
Epoch:  0

Epoch:  1
Loss:  1.6948664
Epoch:  1
Loss:  1.5969706
Epoch:  1
Loss:  1.6950223
Epoch:  1
Loss:  1.6394516
Epoch:  1
Loss:  1.7462744
Epoch:  1
Loss:  1.6367207
Epoch:  1
Loss:  1.5772002
Epoch:  1
Loss:  1.6172491
Epoch:  1
Loss:  1.6755054
Epoch:  1
Loss:  1.6952499
Epoch:  1
Loss:  1.6136014
Epoch:  1
Loss:  1.5283693
Epoch:  1
Loss:  1.5801214
Epoch:  1
Loss:  1.5859351
Epoch:  1
Loss:  1.7144178
Epoch:  1
Loss:  1.6100336
Epoch:  1
Loss:  1.6399885
Epoch:  1
Loss:  1.6243525
Epoch:  1
Loss:  1.7564799
Epoch:  1
Loss:  1.7055752
Epoch:  1
Loss:  1.4691194
Epoch:  1
Loss:  1.6988432
Epoch:  1
Loss:  1.6070197
Epoch:  1
Loss:  1.6878672
Epoch:  1
Loss:  1.6341753
Epoch:  1
Loss:  1.6252457
Epoch:  1
Loss:  1.6221077
Epoch:  1
Loss:  1.6091617
Epoch:  1
Loss:  1.6529462
Epoch:  1
Loss:  1.660382
Epoch:  1
Loss:  1.625864
Epoch:  1
Loss:  1.7039535
Epoch:  1
Loss:  1.6089518
Epoch:  1
Loss:  1.7233918
Epoch:  1
Loss:  1.642257
Epoch:  1
Loss:  1.5572627
Epoch:  1
Loss:  1.6650591
Epoc

KeyboardInterrupt: 

In [46]:
def evaluate(sentence):
    sentence = preprocess_senetence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [47]:
checkpoint.restore(tf.train.latest_checkpoint(''))
evaluate('hace mucho frio aqui.')

('i m not a car . <end> ', '<strat> hace mucho frio aqui . <end>')

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong>Regression</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>